In [ ]:
import torch 
import torch_geometric

torch.seed()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

### Load the Graph

In [ ]:
from generate_graph import graph_to_nx

G = graph_to_nx("IMDB-Movie-Data.csv")

data = torch_geometric.utils.from_networkx(G)

### Generating Graph Embeddings for the model

In [ ]:
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.nn.kge import ComplEx

from gnn import train, lossfn, evaluate

embedding_dim = 64 ## undecided

num_relations = torch.max(data.type).cpu().item() + 1

model = ComplEx(data.num_nodes, num_relations, hidden_channels=embedding_dim).to(device)

print(model)

loader = model.loader(
    head_index= data.edge_index[0],
    rel_type= data.type,
    tail_index= data.edge_index[1],
    batch_size=128,
    shuffle=True,
)

optimizer = optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-6)

In [ ]:
train(model,optimizer,loader)

### Loading and Finetuning the LM

The LM corresponding to the Query Parser

In [ ]:
from lm import ParserLM
lm = ParserLM(to_finetune=True,desc=False)
### This is currently broken , see sparql.ipynb ###